In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from pathlib import Path

import numpy as np
import torch
import torch as tr
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision as tv
from torchvision.transforms import v2
import pandas as pd
import matplotlib.pyplot as plt
import mplcyberpunk

plt.style.use("cyberpunk")

%matplotlib inline

In [3]:
from impl.arch import *

In [4]:
import impl.retinal_dataset as rd

transform = v2.Compose([
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Resize((224, 224)),
])

train_dataset = rd.RetinalDataset(transform=transform, num_rows=-1)

100%|███████████████████████████| 1920/1920 [04:48<00:00,  6.66it/s]


In [5]:
train_dataloader = DataLoader(train_dataset, batch_size=64)

In [ ]:
# plt.figure(figsize=(12,4))
# disease_counts = train_dataset.labels_df.iloc[:, 2:].sum().sort_values(ascending=False)
# disease_counts.plot.bar()

In [6]:
model = Model1()

In [22]:
optimizer = torch.optim.Adam(model.parameters(), lr=1)
loss_fn = nn.CrossEntropyLoss()

In [24]:
model.train()
for i in range(4):
    epoch_loss = 0
    # b = 0

    for images, labels in train_dataloader:
        model.train()

        optimizer.zero_grad()
        logits = model(images)
        loss = loss_fn(logits, labels)
        # if b % 4 == 0:
        #     print(f"{b+1}: {loss/len(images)}")

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        # b += 1

    print(f"==== Epoch {i+1}: train_loss={epoch_loss/len(train_dataset):.10f}")

print("Done")

==== Epoch 1: train_loss=0.0086442397
==== Epoch 2: train_loss=0.0132047426
==== Epoch 3: train_loss=0.0109930342
==== Epoch 4: train_loss=0.0103337455
Done


In [23]:
# torch.save({'model_state': model.state_dict()},'checkpoints/model1_003.chk')

In [ ]:
# checkpoint = torch.load('checkpoints/model1_001.chk')
# model.load_state_dict(checkpoint['model_state'])

In [20]:
@torch.no_grad()
def calculate_loss(dataset):
    dataloader = DataLoader(dataset, batch_size=64)

    total_loss = 0
    for images, labels in dataloader:
        logits = model(images)
        loss = loss_fn(logits, labels)
        total_loss += loss.item()

    total_loss /= len(dataset)
    return total_loss

In [25]:
train_loss = calculate_loss(train_dataset)
train_loss

0.01010064473375678